In [2]:
from preprocessor import *

In [3]:
data = pd.read_csv('~/Documents/datasets/house_prices/train.csv')


prep = Preprocessor()

data = data.drop(['Id'],axis=1)
categorical, numerical = prep.column_types(data)

data[categorical] = data[categorical].fillna("None",axis = 1)
data[numerical] = data[numerical].fillna(0,axis = 1)
print(data.shape)
data.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [155]:
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.svm import SVR


sc = MinMaxScaler()

X_cats = pd.get_dummies(data[categorical])
print(X_cats.shape)
X_cats_df = pd.DataFrame(X_cats)
print(X_cats_df.shape)

X_nums = sc.fit_transform(data[numerical])
print(X_nums.shape)

X_nums_df = pd.DataFrame(X_nums,columns = numerical)
print(X_nums_df.shape)

X_all = pd.concat([X_nums_df,X_cats_df],axis=1)
print(X_all.shape)

X_all.fillna(0,axis = 1,inplace =True)

X = X_all.drop(['SalePrice'],axis=1)

y = X_all['SalePrice']

print("X shape: {}".format(X.shape))
print("Labels shape: {}".format(y.shape))


X_train, X_test, y_train ,y_test = train_test_split(X, y,test_size=0.25, random_state= 42,shuffle=True)

(1460, 267)
(1460, 267)
(1460, 37)
(1460, 37)
(1460, 304)
X shape: (1460, 303)
Labels shape: (1460,)


In [156]:

clf = RandomForestRegressor()
clf.fit(X_train,y_train)

svr = SVR()
svr.fit(X_train,y_train)

print(f'Random Forest accuracy on the training subset: {clf.score(X_train, y_train):.3f}')
print(f'Random Forest accuracy on the testing subset: {clf.score(X_test, y_test):.3f}\n\n')

print(f'SVR accuracy on the training subset: {svr.score(X_train, y_train):.3f}')
print(f'SVR accuracy on the testing subset: {svr.score(X_test, y_test):.3f}')

Random Forest accuracy on the training subset: 0.963
Random Forest accuracy on the testing subset: 0.897


SVR accuracy on the training subset: 0.724
SVR accuracy on the testing subset: 0.703


In [157]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

y_pred_clf = clf.predict(X_test)
y_pred_svr = svr.predict(X_test)

print(f'Mean absolute error RF : {mean_absolute_error(y_test,y_pred_clf):.5f}')
print(f'Mean squared error RF : {mean_squared_error(y_test,y_pred_clf):.5f}')
print(f'R2 Score:            {r2_score(y_test,y_pred_clf):.5f}\n')

print(f'Mean absolute error for SVR : {mean_absolute_error(y_test,y_pred_svr):.5f}')
print(f'Mean squared error for SVR : {mean_squared_error(y_test,y_pred_svr):.5f}')
print(f'R2 Score for SVR :            {r2_score(y_test,y_pred_svr):.5f}')

Mean absolute error RF : 0.02474
Mean squared error RF : 0.00139
R2 Score:            0.89704

Mean absolute error for SVR : 0.04652
Mean squared error for SVR : 0.00401
R2 Score for SVR :            0.70338


In [158]:
# Pickling the trained model
filename = 'housing.pkl'
joblib.dump(clf,filename)

['housing.pkl']

In [160]:
df = pd.read_csv('~/Documents/datasets/house_prices/test.csv')